# COVID-19 Detection on X-Ray Images

## 1. Importing the Training & Test Dataset

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image

## 2. Buiding the CNN Architecture

In [6]:
# CNN Based Model in Keras

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                    

## 3. Data Augmentation, Normalizing Data & Training the Model

In [8]:
# Train from scratch
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_directory(
    '/Users/arnav/Desktop/COVID-19-master/Train',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')


Found 224 images belonging to 2 classes.


In [10]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [11]:
validation_generator = test_dataset.flow_from_directory(
    '/Users/arnav/Desktop/COVID-19-master/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

Found 60 images belonging to 2 classes.


In [12]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=7,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps=2
)

/var/folders/z7/k0sl8h6s2ln4x1nk9r0qg27c0000gn/T/ipykernel_2991/3418722423.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(


Epoch 1/10


2022-11-27 14:16:32.216626: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


7/7 [==============================] - 17s 2s/step - loss: 1.2475 - accuracy: 0.5804 - val_loss: 0.6951 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 18s 3s/step - loss: 0.6574 - accuracy: 0.6116 - val_loss: 0.6431 - val_accuracy: 0.6333
Epoch 3/10
7/7 [==============================] - 17s 2s/step - loss: 0.5213 - accuracy: 0.7723 - val_loss: 0.4565 - val_accuracy: 0.8667
Epoch 4/10
7/7 [==============================] - 18s 3s/step - loss: 0.3588 - accuracy: 0.8438 - val_loss: 0.3621 - val_accuracy: 0.8833
Epoch 5/10
7/7 [==============================] - 17s 2s/step - loss: 0.2853 - accuracy: 0.8795 - val_loss: 0.2463 - val_accuracy: 0.9667
Epoch 6/10
7/7 [==============================] - 17s 2s/step - loss: 0.1910 - accuracy: 0.9062 - val_loss: 0.1226 - val_accuracy: 0.9667
Epoch 7/10
7/7 [==============================] - 18s 3s/step - loss: 0.2181 - accuracy: 0.9196 - val_loss: 0.1057 - val_accuracy: 0.9833
Epoch 8/10
7/7 [=============================

In [14]:
model.save("model_adv.h5")

In [15]:
model.evaluate_generator(train_generator)

/var/folders/z7/k0sl8h6s2ln4x1nk9r0qg27c0000gn/T/ipykernel_2991/326654268.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(train_generator)


[0.11041140556335449, 0.9732142686843872]

In [16]:
model.evaluate_generator(validation_generator)

/var/folders/z7/k0sl8h6s2ln4x1nk9r0qg27c0000gn/T/ipykernel_2991/3378472596.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(validation_generator)


[0.07594632357358932, 0.9666666388511658]

## 4. Model Evaluation

In [17]:
model = load_model('model_adv.h5')

In [18]:
import os

In [19]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [23]:
y_actual = []
y_test = []

In [38]:
from PIL import Image
from tensorflow.keras.utils import load_img, img_to_array 
for i in os.listdir("/Users/arnav/Desktop/COVID-19-master/Val/Normal/"):
  img = image.load_img("/Users/arnav/Desktop/COVID-19-master/Val/Normal/"+i, target_size = (224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict_classes(img)
  y_test.append(p[0,0])
  y_actual.append(1)

AttributeError: module 'keras.preprocessing.image' has no attribute 'load_img'

In [33]:
for i in os.listdir("./CovidDataset/Val/Covid/"):
  img = image.load_img("./CovidDataset/Val/Covid/"+i, target_size = (224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict_classes(img)
  y_test.append(p[0,0])
  y_actual.append(0)

FileNotFoundError: [Errno 2] No such file or directory: './CovidDataset/Val/Covid/'

### 4.1 Confusion Matrix

In [ ]:
y_actual = np.array(y_actual)
y_test = np.array(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_actual, y_test)

In [ ]:
import seaborn as sns
sns.heatmap(cm, cmap=None, annot=True)

### 4.2 Plotting Training & Validation Loss

In [ ]:
import matplotlib.pyplot as plt
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, color='red', label='Training loss')
plt.plot(epochs, val_loss, color='green', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### 4.3 Plotting Training & Validation Accuracy

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
plt.plot(epochs, acc, color='red', label='Training acc')
plt.plot(epochs, val_acc, color='green', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## 5. Some Comparative Results of Radiologist vs CNN Model on Validation set

In [ ]:
class color:
   GREEN = '\033[92m'
   RED = '\033[91m'
   
from PIL import Image
j = 0
for i in os.listdir("./CovidDataset/Val/Normal/"):
  j = j+1
  pil_img = Image.open('./CovidDataset/Val/Normal/'+i)
  im = pil_img
  size = (200,200) #image size
  
  #resize image
  out = im.resize(size)
  display(out)
  img = image.load_img("./CovidDataset/Val/Normal/"+i, target_size = (224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict_classes(img)
  print(color.GREEN + 'Radiologist: COVID-19 -ve' + color.GREEN)

  if p==0:
    print(color.RED + 'CNN Model: COVID-19 +ve' + color.RED)
  else:
    print("CNN Model: COVID-19 -ve")
  if (j==5):
    break

from PIL import Image
j = 0
for i in os.listdir("./CovidDataset/Val/Covid/"):
  j = j+1
  pil_img = Image.open('./CovidDataset/Val/Covid/'+i)
  im = pil_img
  size = (200,200) #image size
  
  #resize image
  out = im.resize(size)
  display(out)
  img = image.load_img("./CovidDataset/Val/Covid/"+i, target_size = (224, 224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict_classes(img)
  print(color.RED + 'Radiologist: COVID-19 +ve' + color.RED)
  if p==0:
    print("CNN Model: COVID-19 +ve")
  else:
    print(color.GREEN + 'Radiologist: COVID-19 -ve' + color.GREEN)
  if (j==5):
    break

*This Method has not been validated clinically and hence should not be used as it is in any clinical computer aided diagnosis system.*

